In [11]:
from langchain.schema import Document
import pandas as pd
import json

# Agora você pode salvar essa lista em um novo arquivo JSON ou processá-la como desejar
with open('../data/json_content_splitted_canal_tavares.json', 'r', encoding='utf-8') as arquivo_json:
    conteudo_json = json.load(arquivo_json)

In [16]:
docs = [Document(metadata=x['metadata'],
                 page_content=x['page_content'],
                 type='Document') for x in conteudo_json]

In [18]:
from langchain_community.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from youtube_transcript_api import YouTubeTranscriptApi
from pytube import Playlist
from tqdm import tqdm

from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

from dotenv import load_dotenv

load_dotenv()

import time
import os


embeddings_size = 3072
embeddings_model = 'text-embedding-3-large'
embeddings = OpenAIEmbeddings(model=embeddings_model)

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

In [19]:
index_name = os.environ.get("PINECONE_INDEX_NAME")

#"pergunte-ao-vc"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
print(f"Existing indexes: {existing_indexes}")

if index_name not in existing_indexes:
    print(f"Index {index_name} does not exist. Please create it first.")
else:
    print(f"Index {index_name} exists. Proceeding.")

Existing indexes: ['tavaresbot']
Index tavaresbot exists. Proceeding.


In [20]:
index = pc.Index(index_name)

documents_splitted = docs
all_docs = documents_splitted
all_docs_to_up = []
for doc in all_docs:
    source = doc.metadata['source']
    query = index.query(
        filter={
            "source": {"$eq": source}
        },
        vector=embeddings.embed_query(doc.page_content),
        top_k=1,
        include_metadata=True
    )
    if len(query['matches']) == 0:
        print('Inserting document: ', doc.metadata['source'])
        all_docs_to_up.append(doc)
print('Total documents to insert: ', len(all_docs_to_up))

if len(all_docs_to_up) > 0:
    docsearch = PineconeVectorStore.from_documents(all_docs_to_up, embeddings, index_name=index_name)

Inserting document:  cQfY8Epzg0s
Inserting document:  cQfY8Epzg0s
Inserting document:  pxULaufWqIE
Inserting document:  JBt5PZz8IAw
Inserting document:  0_ju0-jzBCg
Inserting document:  j0ziVE-rUsY
Inserting document:  deKa6oHrskw
Inserting document:  3awMUpyu7-I
Inserting document:  jrC3ZuV20Wc
Inserting document:  DHNUmOYCJC8
Inserting document:  5xmO3VM8sqQ
Inserting document:  02pd7L9SV6I
Inserting document:  02pd7L9SV6I
Inserting document:  HxeY5_ppUyI
Inserting document:  HxeY5_ppUyI
Inserting document:  RX7EFGNJ-k4
Inserting document:  RX7EFGNJ-k4
Inserting document:  24e3OWHuINQ
Inserting document:  24e3OWHuINQ
Inserting document:  24e3OWHuINQ
Inserting document:  HefbJfUjYQA
Inserting document:  HefbJfUjYQA
Inserting document:  oGNvDo63cD4
Inserting document:  rNDaI4aUTDg
Inserting document:  PrfagKJQeII
Inserting document:  PrfagKJQeII
Inserting document:  Fld0GFo22vo
Inserting document:  Fld0GFo22vo
Inserting document:  tnfBSeBrH80
Inserting document:  76Ya3A-Tn_U
Inserting 